<a href="https://colab.research.google.com/github/Yuliakoi/Speech-Emotion-Recognition/blob/main/%D0%BB%D0%B8%D0%B1%D1%80%D0%BE%D1%81%D0%B0_%D0%BF%D1%80%D0%BE%D1%82%D0%BE%D1%82%D0%B8%D0%BF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install librosa


In [2]:
from google.colab import drive
drive.mount("/drive/")

Mounted at /drive/


In [4]:
dirname = '/drive/My Drive/berlin db'

In [5]:
import librosa
from librosa import display
import sklearn
import pandas as pd
import os
import glob 
import numpy as np

In [6]:
#data, sampling_rate = librosa.load( ФАЙЛ СЮДА)

In [7]:
lst= []
for subdir, dirs, files in os.walk(dirname):
  for file in files:
    try:
      X,sample_rate = librosa.load(os.path.join(subdir,file),res_type='kaiser_fast')
      mfccs = np.mean (librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
      if file[3]=='sa' or file[3]=='su':
        file = file[3:5]
      else:
        file = file[3]
      arr= mfccs,file
      lst.append(arr)
    except ValueError:
      continue

In [8]:
X,y = zip(*lst)

In [9]:
X = np.asarray(X)
y = np.asarray(y)
X.shape, y.shape

((535, 40), (535,))

In [11]:
X_ = pd.DataFrame(data=X)
Y_= pd.DataFrame(data=y)

In [13]:
print(X_)

             0           1          2   ...        37        38        39
0   -228.341934   72.291046 -86.281944  ... -0.110308  0.724302  1.438687
1   -244.960663  103.737282 -46.709778  ... -0.993179 -1.077398 -0.459825
2   -317.613495   77.964516 -75.800743  ...  1.058134 -1.909912 -1.350667
3   -236.031219   98.371170 -69.763718  ...  3.020619  1.185659 -0.548497
4   -291.480621  126.001144 -54.776287  ... -0.789213 -2.046571 -1.820111
..          ...         ...        ...  ...       ...       ...       ...
530 -207.924301   78.814148 -48.295242  ...  1.610538  2.966975  4.417836
531 -243.780609   80.997726 -44.823402  ...  5.715632  4.330856  3.008775
532 -347.895966   58.946327 -73.718117  ...  1.741202  1.733782  2.096647
533 -223.836884   90.293091 -37.966293  ...  2.053149  0.257490  0.309673
534 -274.562164   73.141602 -16.817419  ...  6.865293  5.738996  2.602643

[535 rows x 40 columns]


In [14]:
print(Y_)

     0
0    0
1    0
2    0
3    0
4    0
..  ..
530  0
531  0
532  0
533  0
534  0

[535 rows x 1 columns]


In [15]:
result = pd.concat([Y_,X_], axis=1, ignore_index=True)
result.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40
0,0,-228.341934,72.291046,-86.281944,35.573959,-45.013847,9.084922,-22.116369,-7.248465,-6.220524,-19.505175,-0.960370,-16.618830,1.155036,-6.694911,6.861384,3.023114,12.101799,16.336405,-0.816362,5.351477,0.478905,5.703646,4.788605,5.966914,-0.459323,-2.990561,0.671134,0.036458,2.041339,-1.745802,3.674132,3.847192,2.639743,0.267082,0.241549,1.117174,-1.394378,-0.110308,0.724302,1.438687
1,0,-244.960663,103.737282,-46.709778,37.669445,-29.396229,24.622721,-22.629322,-11.495511,0.063414,-17.077015,2.548406,-6.601529,-1.734473,-5.916827,3.969255,-15.162206,-6.273880,0.527937,-11.648152,2.177778,1.232998,6.537111,4.661454,10.538456,11.840444,8.212301,9.774154,7.678903,9.166143,3.143618,3.841101,3.084623,1.045359,2.930665,3.325513,2.829773,0.241563,-0.993179,-1.077398,-0.459825
2,0,-317.613495,77.964516,-75.800743,45.459934,-44.301662,16.717661,-25.175247,-10.396054,-2.193984,-15.893023,3.315178,-10.978617,2.945383,-9.435846,-0.931071,-12.623302,-5.176109,1.508305,-3.422725,13.840384,10.036343,10.361580,7.013400,10.034327,6.201591,6.347164,5.545530,3.895535,4.852948,-1.394594,2.427273,-0.142525,1.155723,-0.151529,-0.944676,-0.289030,-0.575221,1.058134,-1.909912,-1.350667
3,0,-236.031219,98.371170,-69.763718,38.478413,-22.838850,8.979204,-13.368312,-9.825824,-9.202477,-9.854305,4.302949,-12.147238,2.761482,-7.477390,0.953241,-3.708987,-7.027302,-0.444048,-7.894135,-4.130747,-4.413131,-0.183898,-5.776920,-0.503310,1.514138,-1.476832,2.176158,-1.468371,0.193033,3.018370,4.966105,0.783588,0.553935,0.850555,0.480453,3.458095,2.369641,3.020619,1.185659,-0.548497
4,0,-291.480621,126.001144,-54.776287,44.369537,-15.479568,-8.542458,-3.939008,-1.768697,-23.222383,-6.206619,10.038188,-16.813200,2.276435,-4.781696,0.917384,-5.758445,-7.096720,1.332785,-7.849123,0.385087,-4.396211,-0.247892,-1.839953,-4.661735,-0.774366,-3.924490,-2.735914,-2.658870,-2.271216,-4.969137,-1.874449,-1.183285,-2.986231,-1.151088,-0.733136,-2.125716,-3.368592,-0.789213,-2.046571,-1.820111


In [ ]:
#result.to_csv('/drive/My Drive/мага речевые технологии/cvs датасетов opensmile/librosa_savee.csv', index=False, header=False)

In [16]:
import time

In [17]:
#импорт всех моделей
from sklearn.linear_model import LogisticRegression, RidgeClassifier, SGDClassifier, PassiveAggressiveClassifier, Perceptron

from sklearn.svm import SVC

from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

In [18]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

In [19]:
models = {
  'log': LogisticRegression(),
  'sgd': SGDClassifier(),
  'ridge': RidgeClassifier(),
  'pac': PassiveAggressiveClassifier(),
  'perc': Perceptron(),

  'linear_svc': SVC(kernel='linear'),
  'rbf_svc': SVC(kernel='rbf'),
  'poly_svc': SVC(kernel='poly'),

  'mlp': MLPClassifier(),
  'knn': KNeighborsClassifier(),
  'gpc': GaussianProcessClassifier(),
  'qda': QuadraticDiscriminantAnalysis(),

  'gnb': GaussianNB(),
  'bnb': BernoulliNB(),

  'tree': DecisionTreeClassifier(),

  'forest': RandomForestClassifier(),
  'ada': AdaBoostClassifier(),
  'extra': ExtraTreesClassifier(),
  'gboost': GradientBoostingClassifier(),
}
print(len(models))

19


In [20]:
from sklearn.model_selection import train_test_split
import sklearn.metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

def test_model(model, x, y):  
  x_train, x_test, y_train, y_test = train_test_split(X,y, test_size=0.3, stratify = y)
  
  pipe = Pipeline([
                 ('scaler', StandardScaler()), 
                 ('cls', models[model])
                 ])
  start_time = time.time()
  pipe.fit(x_train, y_train)
  time_fit="%s sec" % (time.time() - start_time)
  #fit для тестовой и тренировочной выборки
  y_test_pred =pipe.predict(x_test)
  y_train_pred =pipe.predict(x_train)
  #accuracy для предсказанных значений по test и train
  acc_test = accuracy_score(y_test, y_test_pred)
  acc_train = accuracy_score(y_train, y_train_pred)
  prec_test = precision_score(y_test, y_test_pred, average=None).mean()
  recall_test = recall_score(y_test, y_test_pred, average=None).mean()
  #precision & recall для test
  #prec_test = precision_score(y_test, y_test_pred)
  #recall_test = recall_score(y_test, y_test_pred, average='macro')


  

  
  scores = cross_val_score(pipe, x, y, cv=5).mean()

  return {
      "MODEL": model,
      "SCORE":scores,
      "ACCURACY TRAIN":acc_train,
      "ACCURACY TEST":acc_test,
      "PRECISION TEST":prec_test,
      "RECALL TEST":recall_test,
      "TIME FOR FIT":time_fit 
      }

In [21]:
test_model('forest', X, y)

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'ACCURACY TEST': 0.9006211180124224,
 'ACCURACY TRAIN': 1.0,
 'MODEL': 'forest',
 'PRECISION TEST': 0.4503105590062112,
 'RECALL TEST': 0.5,
 'SCORE': 0.8990654205607477,
 'TIME FOR FIT': '0.3978126049041748 sec'}

In [22]:
for name, model in models.items():
  res = test_model(name, X, y)
  #table=pd.DataFrame.from_dict(res)
  print(name, res)

log {'MODEL': 'log', 'SCORE': 0.8411214953271028, 'ACCURACY TRAIN': 0.9144385026737968, 'ACCURACY TEST': 0.9006211180124224, 'PRECISION TEST': 0.7054140127388535, 'RECALL TEST': 0.555603448275862, 'TIME FOR FIT': '0.05456948280334473 sec'}
sgd {'MODEL': 'sgd', 'SCORE': 0.805607476635514, 'ACCURACY TRAIN': 0.9278074866310161, 'ACCURACY TEST': 0.8757763975155279, 'PRECISION TEST': 0.606953642384106, 'RECALL TEST': 0.5696120689655173, 'TIME FOR FIT': '0.007169008255004883 sec'}


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


ridge {'MODEL': 'ridge', 'SCORE': 0.8915887850467291, 'ACCURACY TRAIN': 0.9090909090909091, 'ACCURACY TEST': 0.9006211180124224, 'PRECISION TEST': 0.4503105590062112, 'RECALL TEST': 0.5, 'TIME FOR FIT': '0.05728769302368164 sec'}
pac {'MODEL': 'pac', 'SCORE': 0.7700934579439251, 'ACCURACY TRAIN': 0.8743315508021391, 'ACCURACY TEST': 0.84472049689441, 'PRECISION TEST': 0.5759803921568627, 'RECALL TEST': 0.5801724137931035, 'TIME FOR FIT': '0.012067317962646484 sec'}
perc {'MODEL': 'perc', 'SCORE': 0.7700934579439253, 'ACCURACY TRAIN': 0.9010695187165776, 'ACCURACY TEST': 0.8385093167701864, 'PRECISION TEST': 0.5859929078014184, 'RECALL TEST': 0.6045258620689655, 'TIME FOR FIT': '0.004138946533203125 sec'}


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


linear_svc {'MODEL': 'linear_svc', 'SCORE': 0.8261682242990653, 'ACCURACY TRAIN': 0.9037433155080213, 'ACCURACY TEST': 0.9006211180124224, 'PRECISION TEST': 0.4503105590062112, 'RECALL TEST': 0.5, 'TIME FOR FIT': '0.07249879837036133 sec'}
rbf_svc {'MODEL': 'rbf_svc', 'SCORE': 0.9028037383177571, 'ACCURACY TRAIN': 0.9037433155080213, 'ACCURACY TEST': 0.9006211180124224, 'PRECISION TEST': 0.4503105590062112, 'RECALL TEST': 0.5, 'TIME FOR FIT': '0.00868368148803711 sec'}
poly_svc {'MODEL': 'poly_svc', 'SCORE': 0.8878504672897197, 'ACCURACY TRAIN': 0.93048128342246, 'ACCURACY TEST': 0.8819875776397516, 'PRECISION TEST': 0.44936708860759494, 'RECALL TEST': 0.4896551724137931, 'TIME FOR FIT': '0.008612394332885742 sec'}


...

mlp {'MODEL': 'mlp', 'SCORE': 0.8074766355140186, 'ACCURACY TRAIN': 0.9919786096256684, 'ACCURACY TEST': 0.8944099378881988, 'PRECISION TEST': 0.6551282051282051, 'RECALL TEST': 0.5521551724137931, 'TIME FOR FIT': '0.8402073383331299 sec'}
knn {'MODEL': 'knn', 'SCORE': 0.8953271028037383, 'ACCURACY TRAIN': 0.9117647058823529, 'ACCURACY TEST': 0.8819875776397516, 'PRECISION TEST': 0.44936708860759494, 'RECALL TEST': 0.4896551724137931, 'TIME FOR FIT': '0.0146942138671875 sec'}
gpc {'MODEL': 'gpc', 'SCORE': 0.8485981308411216, 'ACCURACY TRAIN': 1.0, 'ACCURACY TEST': 0.8385093167701864, 'PRECISION TEST': 0.49133109619686804, 'RECALL TEST': 0.49331896551724136, 'TIME FOR FIT': '0.16550540924072266 sec'}
qda {'MODEL': 'qda', 'SCORE': 0.9028037383177571, 'ACCURACY TRAIN': 1.0, 'ACCURACY TEST': 0.9006211180124224, 'PRECISION TEST': 0.4503105590062112, 'RECALL TEST': 0.5, 'TIME FOR FIT': '0.039699554443359375 sec'}
gnb {'MODEL': 'gnb', 'SCORE': 0.8, 'ACCURACY TRAIN': 0.8556149732620321, 'ACCUR

/usr/local/lib/python3.7/dist-packages/sklearn/discriminant_analysis.py:691: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


bnb {'MODEL': 'bnb', 'SCORE': 0.874766355140187, 'ACCURACY TRAIN': 0.9090909090909091, 'ACCURACY TEST': 0.906832298136646, 'PRECISION TEST': 0.953125, 'RECALL TEST': 0.53125, 'TIME FOR FIT': '0.0036783218383789062 sec'}
tree {'MODEL': 'tree', 'SCORE': 0.7813084112149532, 'ACCURACY TRAIN': 1.0, 'ACCURACY TEST': 0.8260869565217391, 'PRECISION TEST': 0.5142241379310345, 'RECALL TEST': 0.5142241379310345, 'TIME FOR FIT': '0.020038366317749023 sec'}


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


forest {'MODEL': 'forest', 'SCORE': 0.8990654205607477, 'ACCURACY TRAIN': 1.0, 'ACCURACY TEST': 0.9006211180124224, 'PRECISION TEST': 0.4503105590062112, 'RECALL TEST': 0.5, 'TIME FOR FIT': '0.40036487579345703 sec'}
ada {'MODEL': 'ada', 'SCORE': 0.8448598130841122, 'ACCURACY TRAIN': 0.9946524064171123, 'ACCURACY TEST': 0.84472049689441, 'PRECISION TEST': 0.5296257796257796, 'RECALL TEST': 0.5245689655172414, 'TIME FOR FIT': '0.21439623832702637 sec'}
extra {'MODEL': 'extra', 'SCORE': 0.9028037383177571, 'ACCURACY TRAIN': 1.0, 'ACCURACY TEST': 0.8944099378881988, 'PRECISION TEST': 0.45, 'RECALL TEST': 0.496551724137931, 'TIME FOR FIT': '0.18318843841552734 sec'}
gboost {'MODEL': 'gboost', 'SCORE': 0.8485981308411216, 'ACCURACY TRAIN': 1.0, 'ACCURACY TEST': 0.9006211180124224, 'PRECISION TEST': 0.7028301886792453, 'RECALL TEST': 0.527801724137931, 'TIME FOR FIT': '0.5493454933166504 sec'}


In [23]:
table = pd.DataFrame()
for name, _ in models.items():
  res = pd.DataFrame(test_model(name, X, y), index=[name, ])
  table = pd.concat([table, res], axis=0)
table.head()

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations 

,MODEL,SCORE,ACCURACY TRAIN,ACCURACY TEST,PRECISION TEST,RECALL TEST,TIME FOR FIT
log,log,0.841121,0.925134,0.844720,0.495455,0.496767,0.04922938346862793 sec
sgd,sgd,0.794393,0.914439,0.844720,0.575980,0.580172,0.004396200180053711 sec
ridge,ridge,0.891589,0.909091,0.900621,0.450311,0.500000,0.011863946914672852 sec
pac,pac,0.790654,0.847594,0.832298,0.563010,0.573276,0.008141756057739258 sec
perc,perc,0.770093,0.922460,0.857143,0.544242,0.531466,0.0045282840728759766 sec


In [24]:
table.drop('MODEL', axis=1, inplace=True)

In [25]:
table

,SCORE,ACCURACY TRAIN,ACCURACY TEST,PRECISION TEST,RECALL TEST,TIME FOR FIT
log,0.841121,0.925134,0.844720,0.495455,0.496767,0.04922938346862793 sec
sgd,0.794393,0.914439,0.844720,0.575980,0.580172,0.004396200180053711 sec
ridge,0.891589,0.909091,0.900621,0.450311,0.500000,0.011863946914672852 sec
pac,0.790654,0.847594,0.832298,0.563010,0.573276,0.008141756057739258 sec
perc,0.770093,0.922460,0.857143,0.544242,0.531466,0.0045282840728759766 sec
linear_svc,0.826168,0.925134,0.850932,0.447712,0.472414,0.037218570709228516 sec
rbf_svc,0.902804,0.903743,0.900621,0.450311,0.500000,0.009350061416625977 sec
poly_svc,0.887850,0.930481,0.894410,0.619198,0.524353,0.010329246520996094 sec
mlp,0.807477,0.986631,0.900621,0.702830,0.527802,0.8052444458007812 sec
knn,0.895327,0.909091,0.888199,0.449686,0.493103,0.003521442413330078 sec


In [ ]:
#table.to_csv('/drive/My Drive/мага речевые технологии/cvs датасетов opensmile/librosa_savee_метрики.csv')